### Задача
Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Нужно обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности правок.

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
movie_data = pd.read_csv('toxic_comments.csv')
X, y = movie_data['text'], movie_data['toxic']

In [3]:
import nltk
nltk.download('wordnet')

documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=3950, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=3950, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(documents).toarray()

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
from sklearn.metrics import f1_score

In [8]:
from catboost import CatBoostClassifier

In [15]:
modelCatBoost = CatBoostClassifier(learning_rate=1, loss_function= 'Logloss', depth=6, iterations=300)
modelCatBoost = modelCatBoost.fit(X_train, y_train)

0:	learn: 0.2685910	total: 2.43s	remaining: 12m 6s
1:	learn: 0.2316131	total: 3.93s	remaining: 9m 45s
2:	learn: 0.2122856	total: 5.08s	remaining: 8m 22s
3:	learn: 0.2009749	total: 6.18s	remaining: 7m 37s
4:	learn: 0.1934919	total: 7.25s	remaining: 7m 7s
5:	learn: 0.1874544	total: 8.35s	remaining: 6m 49s
6:	learn: 0.1819223	total: 9.41s	remaining: 6m 34s
7:	learn: 0.1751470	total: 10.6s	remaining: 6m 25s
8:	learn: 0.1717726	total: 11.6s	remaining: 6m 16s
9:	learn: 0.1683868	total: 12.7s	remaining: 6m 9s
10:	learn: 0.1656855	total: 13.8s	remaining: 6m 2s
11:	learn: 0.1629861	total: 14.9s	remaining: 5m 56s
12:	learn: 0.1605479	total: 15.9s	remaining: 5m 52s
13:	learn: 0.1582806	total: 17.1s	remaining: 5m 48s
14:	learn: 0.1554632	total: 18.2s	remaining: 5m 46s
15:	learn: 0.1532153	total: 19.3s	remaining: 5m 43s
16:	learn: 0.1516883	total: 20.4s	remaining: 5m 39s
17:	learn: 0.1494842	total: 21.6s	remaining: 5m 37s
18:	learn: 0.1483623	total: 22.6s	remaining: 5m 34s
19:	learn: 0.1469032	tota

In [16]:
y_pred = modelCatBoost.predict(X_test)

In [18]:
print(f1_score(y_test,y_pred))

0.750208576672785


### Преобразование корпусов комментариев со следующими параметрами: max_features=3950, min_df=5, max_df=0.7, stop_words=stopwords.words('english')

### Модель CatBoostClassifier со значением метрики качества *F1* 0.750208576672785 достигнуто при параметрах learning_rate=1, loss_function= 'Logloss', depth=6, iterations=300